**Phân tích cảm xúc là việc giải thích và phân loại cảm xúc (tích cực, tiêu cực và trung tính) trong dữ liệu văn bản bằng kỹ thuật phân tích văn bản.  Phân tích tình cảm cho phép các doanh nghiệp xác định cảm xúc của khách hàng đối với sản phẩm, thương hiệu hoặc dịch vụ trong các cuộc trò chuyện và phản hồi trực tuyến.**

# Ưu điểm của việc phân tích cảm xúc
- Bằng cách sử dụng phân tích cảm xúc, bạn đánh giá cảm nhận của khách hàng về các lĩnh vực khác nhau của doanh nghiệp mà không cần phải đọc hàng nghìn nhận xét của khách hàng cùng một lúc.
- Nếu bạn có hàng nghìn hoặc thậm chí hàng chục nghìn câu trả lời khảo sát mỗi tháng, thì không thể có một người đọc tất cả các câu trả lời này và có một thước đo không khách quan và nhất quán về tình cảm của khách hàng.  Bằng cách sử dụng phân tích tình cảm và tự động hóa quy trình này, bạn có thể dễ dàng đi sâu vào các phân khúc khách hàng khác nhau của doanh nghiệp mình và hiểu rõ hơn về tình cảm trong các phân khúc này.

# Nhược điểm của việc sử dụng phân tích cảm xúc
- Mặc dù phân tích cảm xúc là hữu ích, nhưng chúng tôi không tin rằng nó là một sự thay thế hoàn toàn cho việc đọc các câu trả lời khảo sát, vì bản thân các nhận xét thường có những sắc thái hữu ích.  Việc phân tích cảm xúc có thể giúp bạn thêm ở đâu là xác định bạn nên đọc những nhận xét nào trong số những nhận xét này, chẳng hạn như cho phép bạn tập trung vào những nhận xét tiêu cực nhất.

# Cách phân tích cảm xúc thực sự hoạt động
- Phân tích tình cảm truyền thống bao gồm việc sử dụng các từ điển tham khảo về mức độ tích cực của các từ nhất định và sau đó tính điểm trung bình của những điểm này như cảm xúc của văn bản đó.
- Bước tiếp theo từ đây là sử dụng một mô hình ML đơn giản để phân loại.  Điều này được thực hiện bằng cách tạo ra các "đặc trưng" từ văn bản sau đó sử dụng các đặc trưng này để dự đoán một "nhãn".  Một ví dụ về việc tạo các tính năng là chia nhỏ văn bản thành các từ và sau đó sử dụng các từ này và tần số của chúng trong văn bản làm đặc truwng.

In [2]:
import numpy as np 
import pandas as pd 

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.layers import Dropout

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
train_df = pd.read_csv('./data/train.csv')

In [4]:
train_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
# Drop Nan Values
X = train_df.dropna()

# Get training data
X = train_df.drop('sentiment', axis=1)

# Get target label
y = train_df['sentiment']

# Onehot Representation

In [6]:
# Onehot Representation

messages = X.copy()

# The reset_index() function is used to generate a new DataFrame or Series with the index reset
messages.reset_index(inplace = True)

# Downloading stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/thang/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# Dataset Preprocessing
ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):
    # replace with space words other than a-1, A-Z
    
    review = re.sub('[^a-zA-Z]', ' ', str(messages['text'][i]))
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [8]:
voc_size = 5000

In [9]:
# One Hot Encoding
onehot_repr = [one_hot(words, voc_size) for words in corpus]

# Embedding Representation

In [10]:
# making all sentences of same length
sent_length = 30
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)

In [11]:
# Finding the numberof labels
num_labels = len(set(train_df['sentiment']))

In [12]:
num_labels

3

# Constructing LSTM model

In [13]:
embedding_vector_features = 40

## Creating model
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(num_labels,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 256,703
Trainable params: 256,703
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
import tensorflow as tf
from sklearn import preprocessing

# encode label to int
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

X_final = np.array(embedded_docs)
y_final = np.array(y)

from tensorflow.keras.utils import to_categorical
y_final = to_categorical(y_final)

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [18]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
288/288 [==============================] - 2s 8ms/step - loss: 0.9000 - accuracy: 0.5752 - val_loss: 0.7858 - val_accuracy: 0.6595
Epoch 2/10
288/288 [==============================] - 2s 7ms/step - loss: 0.7252 - accuracy: 0.6955 - val_loss: 0.7604 - val_accuracy: 0.6741
Epoch 3/10
288/288 [==============================] - 2s 7ms/step - loss: 0.6729 - accuracy: 0.7234 - val_loss: 0.7583 - val_accuracy: 0.6741
Epoch 4/10
288/288 [==============================] - 2s 7ms/step - loss: 0.6417 - accuracy: 0.7384 - val_loss: 0.7668 - val_accuracy: 0.6754
Epoch 5/10
288/288 [==============================] - 2s 7ms/step - loss: 0.6210 - accuracy: 0.7514 - val_loss: 0.7708 - val_accuracy: 0.6747
Epoch 6/10
288/288 [==============================] - 2s 7ms/step - loss: 0.6013 - accuracy: 0.7597 - val_loss: 0.7923 - val_accuracy: 0.6664
Epoch 7/10
288/288 [==============================] - 2s 7ms/step - loss: 0.5829 - accuracy: 0.7673 - val_loss: 0.7875 - val_accuracy: 0.6723
Epoch 

# Putting all the pre-processing steps done for training in one function

In [19]:
def return_x_y(X):
    
    # Drop Nan Values
    X = X.fillna(0)
    
    messages = X.copy()

    messages.reset_index(inplace = True)

    # Dataset Preprocessing
    ps = PorterStemmer()

    corpus = []

    for i in range(0, len(messages)):
        # replace with space words other than a-1, A-Z

        review = re.sub('[^a-zA-Z]', ' ', str(messages['text'][i]))
        review = review.lower()
        review = review.split()

        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)

    # vocabulray size
    voc_size = 5000

    onehot_repr = [one_hot(words, voc_size) for words in corpus]

    # Embedding Representation
    # making all sentences of same length
    sent_length = 30
    embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)

    X_final = np.array(embedded_docs)
    
    
    return X_final, X

In [20]:
# reading test data and pre-processing
test_df = pd.read_csv('./data//test.csv')
X_test,X_test_drop = return_x_y(test_df)

In [21]:
# making prediction
y_pred_test = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [22]:
y_pred_test

array([1, 2, 0, ..., 0, 2, 2])

In [23]:
submission_data = pd.read_csv('./data/sample_submission.csv')

In [24]:
len(X_test_drop['textID']), len(y_pred_test)

(3534, 3534)

In [25]:
df_sub = pd.DataFrame()
df_sub['id'] = X_test_drop['textID']
df_sub['text'] = X_test_drop['text']
df_sub['sentiment_predicted'] = le.inverse_transform(y_pred_test)
df_sub['sentiment_actual'] = X_test_drop['sentiment']

In [ ]:
df_sub.to_csv('gender_submission.csv', index=False)